In [1]:
# Import the necessary modules
from pyspark.sql.functions import *
from ultis import *
from datetime import datetime
from decouple import config

spark = create_spark()

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.postgresql#postgresql added as a dependency
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a164668f-68bf-4d77-9b3c-b6f2c05a30a9;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;11.2.2.jre17 in central
:: resolution report :: resolve 1189ms :: artifacts dl 46ms
	:: modules i

In [2]:
table = "USER_RATING_FACT"

In [3]:
df = spark.sql(f'''
SELECT
    *
FROM delta.`s3a://gold/{table}`
    ''')

23/10/07 06:33:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
server_url = "sqlserver:1433"
database_name = "model"
table_name = f"report.{table}"
username = "sa"
password = "@Random1234567890"
jdbcUrl = f"jdbc:sqlserver://{server_url};databaseName={database_name};encrypt=true;trustServerCertificate=true"
jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
try:
    (df.write.format("jdbc")
    .option("url", jdbcUrl)
    .option("dbtable", table_name)
    .option("user", username)
    .option("password", password)
    .option("driver", jdbcDriver)
    .mode("overwrite")
    .save())
except ValueError as error :
    print("Connector write failed", error)

23/10/07 06:33:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
spark.stop()